In [14]:
%%capture
%pip install plotly

In [15]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, FloatSlider, IntSlider

# Inverse Fourier Transform

We will be performing inverse fourier transform on a rectangular wave in this notebook. <br>

The rectangular function is defined as: <br>
$$
\text{rect}\left(\frac{f}{B}\right) =
\begin{cases}
1, & |f| \leq \frac{B}{2} \\
0, & |f| > \frac{B}{2}
\end{cases}
$$
where:
- \( f \) is the frequency variable,  
- \( B \) is the total bandwidth,  
- \( $\text{rect}(\cdot)$ \) is the rectangular (boxcar) function that is 1 within the interval and 0 elsewhere.

The above function is also known as a boxcar function and it describes a frequency-domain representation of a rectangular wave. This function helps mask the bandlimited rectangular wave having a frequency range $-\frac{B}{2} \le f \le \frac{B}{2}$. <br>

Inverse fourier transform of the above function would result in a sinc function in time domain: <br>
$$
\mathcal{F}^{-1} \left\{ \text{rect}\left( \frac{f}{B} \right) \right\} = B \cdot \text{sinc}(B t)
$$

where:
- \( f \) is the frequency variable,  
- \( t \) is the time variable,  
- \( B \) is the total bandwidth,  
- \( $\text{rect}(\cdot)$ \) is the rectangular (boxcar) function that is 1 within the interval and 0 elsewhere,  
- \( $\text{sinc}(Bt)$ = $\dfrac{\sin(\pi B t)}{\pi B t}$ \) is the normalised sinc function.


## Frequency Domain 
### Boxcar Function

In [16]:
# Boxcar (rectangular) function
def rect(f, B):
    return np.where(np.abs(f) <= B/2, 1, 0)

def plot_rect(min_freq, max_freq, bandwidth):
    resolution = 100 # Points per unit ( per hz )

    # Dynamically compute number of samples
    num_points = int((max_freq-min_freq) * resolution)

    # Frequency axis
    if bandwidth/2 > max_freq:
        bandwidth = max_freq - min_freq
    f = np.linspace(min_freq, max_freq, num_points)

    # Evaluate the rect function
    y = rect(f, bandwidth)

    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(f, y, label=fr'$\text{{rect}}\left(\frac{{f}}{{{bandwidth}}}\right)$')
    plt.axvline(-bandwidth/2, color='r', linestyle='--', label=fr'$-B/2$')
    plt.axvline(bandwidth/2, color='g', linestyle='--', label=fr'$B/2$')
    plt.title("Boxcar Function (Rectangular Window)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.ylim(-0.2, 1.2)
    plt.legend()
    plt.show()

In [17]:
# Sliders
min_freq_slider = IntSlider(min=-50, max=0, step=1, value=-10, description='Min Freq')
max_freq_slider = IntSlider(min=0, max=50, step=1, value=10, description='Max Freq')
bandwidth_slider = IntSlider(min=1, max=100, step=1, value=10, description='Bandwidth')

def update_bandwidth_range(*args):
    min_f = min_freq_slider.value
    max_f = max_freq_slider.value
    max_bw = max(1, max_f - min_f)
    bandwidth_slider.max = max_bw
    if bandwidth_slider.value > max_bw:
        bandwidth_slider.value = max_bw


min_freq_slider.observe(update_bandwidth_range, names='value')
max_freq_slider.observe(update_bandwidth_range, names='value')


# Interactive UI
ui = interactive(
    plot_rect,
    min_freq=min_freq_slider,
    max_freq=max_freq_slider,
    bandwidth=bandwidth_slider
)
display(ui)

interactive(children=(IntSlider(value=-10, description='Min Freq', max=0, min=-50), IntSlider(value=10, descri…

## Fourier Transforms
Fourier transforms links signals between the time and frequency domain by breaking down the signal into its complex signals (in the complex domain).

### Forward Fourier Transform
The **Fourier Transform** converts a time-domain signal \( $x(t)$ \) into its frequency-domain representation \( $X(f)$ \):
$$
X(f) = \int_{-\infty}^{\infty} x(t) \, e^{-j 2\pi f t} \, dt
$$
- \( $x(t)$ \): Time-domain signal  
- \( $X(f)$ \): Frequency-domain spectrum (complex-valued)  
- \( $f$ \): Frequency in Hz  
- \( $e^{-j 2\pi f t}$ \): Complex exponential basis

### Inverse Fourier Transform
The **Inverse Fourier Transform** reconstructs the time-domain signal \( $x(t)$ \) from its frequency-domain representation \( $X(f)$ \):

$$
x(t) = \int_{-\infty}^{\infty} X(f) \, e^{j 2\pi f t} \, df
$$

- Synthesises all frequency components into the original time-domain signal  
- \( $e^{+j 2\pi f t}$ \): Opposite rotation to undo the forward transform

## Time Domain
### Sinc Function 
We now perform inverse fourier transform on the boxcar function. This results in the following equation:
$$
B sinc(Bt)
$$
- \( $B$ \): Bandwidth
- \( $t$ \): Time independent variable




In [18]:
def sinc(B, t):
    """Sinc function corresponding to rect(f/B)."""
    return B * np.sinc(B * t)  # np.sinc(x) = sin(pi*x)/(pi*x)

# Plotting function
def plot_sinc(time_min, time_max, bandwidth, resolution, amplitude_min, amplitude_max):
    # Compute number of points
    num_points = int((time_max - time_min) * resolution)
    if num_points <= 1:
        print("Time window too small for given resolution.")
        return

    # Time axis
    t = np.linspace(time_min, time_max, num_points)
    
    # Compute sinc
    y = sinc(bandwidth, t)

    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(t, y, label=fr'$B \cdot \text{{sinc}}(B t),\ B={bandwidth}$')
    plt.title("Inverse Fourier Transform of rect: Sinc Function")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.axvline(0, color='k', linestyle='--', linewidth=0.8)
    plt.legend()
    plt.ylim(amplitude_min, amplitude_max)
    plt.show()

# Define sliders
sliders = {
    "time_min": FloatSlider(min=-5, max=0, step=0.1, value=-2, description="Min Time"),
    "time_max": FloatSlider(min=0.1, max=5, step=0.1, value=2, description="Max Time"),
    "bandwidth": IntSlider(min=1, max=20, step=1, value=5, description="Bandwidth"),
    "resolution": IntSlider(min=50, max=1000, step=50, value=500, description="Res"),
    "amplitude_min": FloatSlider(min=-10.0, max=0.0, step=0.1, value=-5.0, description="Min Amplitude"),
    "amplitude_max": FloatSlider(min=0.0, max=20.0, step=0.1, value=10.0, description="Max Amplitude"),
}

# Create and display interactive UI
ui = interactive(plot_sinc, **sliders)
display(ui)


interactive(children=(FloatSlider(value=-2.0, description='Min Time', max=0.0, min=-5.0), FloatSlider(value=2.…

In [19]:
def plot_boxcar_and_sinc(f_s, duration, bandwidth):
    # Number of samples
    N = int(f_s * duration)
    if N % 2 != 0:
        N += 1  # Ensure even length for symmetry

    # Frequency axis (Hz)
    freqs = np.fft.fftfreq(N, d=1/f_s)
    freqs_shifted = np.fft.fftshift(freqs)

    # Boxcar function in frequency domain
    boxcar = np.where(np.abs(freqs_shifted) <= bandwidth / 2, 1.0, 0.0)

    # Prepare for IFFT
    boxcar_ifft = np.fft.ifftshift(boxcar)
    time_signal = np.fft.ifft(boxcar_ifft)
    time_signal = np.real(time_signal)

    # Time axis
    t = np.linspace(-duration / 2, duration / 2, N, endpoint=False)
    t_shifted = np.fft.fftshift(t)
    time_signal_shifted = np.fft.fftshift(time_signal)

    # --- Frequency-Domain Plot ---
    fig_freq = go.Figure()
    fig_freq.add_trace(go.Scatter(x=freqs_shifted, y=boxcar, mode='lines',
                                  name='Boxcar', line=dict(color='royalblue')))
    fig_freq.update_layout(
        title="Boxcar in Frequency Domain",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Time-Domain Plot ---
    fig_time = go.Figure()
    fig_time.add_trace(go.Scatter(x=np.fft.fftshift(t_shifted), y=time_signal_shifted, mode='lines',
                                  name='Sinc', line=dict(color='firebrick')))
    fig_time.update_layout(
        title="Sinc from Inverse FFT of Boxcar",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # Show both plots
    fig_freq.show()
    fig_time.show()

In [20]:
# Interactive UI
ui = interactive(
    plot_boxcar_and_sinc,
    f_s=IntSlider(min=100, max=1000, step=10, value=250, description="Sampling Freq (Hz)"),
    duration=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description="Duration (s)"),
    bandwidth=IntSlider(min=10, max=1000, step=10, value=100, description="Bandwidth (Hz)")
)
display(ui)

interactive(children=(IntSlider(value=250, description='Sampling Freq (Hz)', max=1000, min=100, step=10), Floa…

## Nyquist Frequency
Recall that the Nyquist frequency has the equation:
$$
f_{Nyquist} = \frac{f_s}{2}
$$
where
- $f_s$ is the **sampling frequeny** (in Hz)
- $f_{Nyquist}$ is the **maximum frequency** that can be accurately represented when sampling a signal

If a signal contains frequency components above $\frac{f_s}{2}$ they will be aliased (folded back into the lower frequency range, distorting the original signal)

If we look at the following expression as plotted above: $rect(\frac{f}{B})$, <br>
we note that it is non-zero for $f \in [-\frac{B}{2}, \frac{B}{2}]$. <br>

This means that the signal is bandlimited to $\pm \frac{B}{2} \text{Hz}$ with a total bandwidth of $B$ Hz. <br>
Over here, the highest frequency present in the signal is $f = \frac{B}{2}$

To sample and reconstruct a bandlimited signal with maximum frequency $f_{max}$, the sampling frequency must satisfy (according to the Nyquist sampling theorem): 
$$
f_s \ge 2 f_{max}
$$

Therefore we have,
$f_{max} = \frac{B}{2} \implies f_s \ge B$

The above states the minimum sampling frequency that avoids aliasing for a signal bandlimited to $\pm \frac{B}{2}$

**NOTE:** *The above plot can be adjusted to see the aliasing effect when we set* $B > f_s$

## Adding Linear Phase Shift
Linear phase shifts can be added to the signal in frequency domain. This phase shift translates to a translation in the time domain which we will see below.

In [21]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import IntSlider, FloatSlider, interactive
from IPython.display import display

def plot_shifted_sinc(f_s, duration, bandwidth, time_shift):
    N = int(f_s * duration)
    if N % 2 != 0:
        N += 1  # ensure even length

    # Frequency axis (Hz)
    freqs = np.fft.fftfreq(N, d=1/f_s)
    freqs_shifted = np.fft.fftshift(freqs)

    # Frequency domain: boxcar with linear phase
    boxcar = np.where(np.abs(freqs_shifted) <= bandwidth / 2, 1.0, 0.0)
    linear_phase = np.exp(1j * 2 * np.pi * freqs_shifted * time_shift)
    freq_shifted = boxcar * linear_phase

    # Inverse FFT to get time-domain sinc
    time_signal = np.fft.ifft(np.fft.ifftshift(freq_shifted))
    time_signal = np.real(time_signal)  # discard imaginary part due to numerics

    # Time axis
    t = np.linspace(-duration / 2, duration / 2, N, endpoint=False)

    # --- Frequency Magnitude Plot ---
    fig_mag = go.Figure()
    fig_mag.add_trace(go.Scatter(x=freqs_shifted,
                                 y=np.abs(freq_shifted),
                                 mode='lines',
                                 name='|X(f)|',
                                 line=dict(color='royalblue')))
    fig_mag.update_layout(
        title="Frequency Domain: Magnitude",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Frequency Phase Plot ---
    fig_phase = go.Figure()
    fig_phase.add_trace(go.Scatter(x=freqs_shifted,
                                   y=np.angle(freq_shifted),
                                   mode='lines',
                                   name='∠X(f)',
                                   line=dict(color='purple')))
    fig_phase.update_layout(
        title="Frequency Domain: Phase",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Phase (radians)",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Time Domain Plot (Shifted Sinc) ---
    fig_time = go.Figure()
    fig_time.add_trace(go.Scatter(x=t,
                                  y=np.fft.fftshift(time_signal),
                                  mode='lines',
                                  name='Sinc(t - t₀)',
                                  line=dict(color='darkorange')))
    fig_time.update_layout(
        title=f"Time Domain: Sinc Shifted by t_0 = {time_shift}s",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Frequency Phase Plot (unwrapped) ---
    # Shows the true frequency plot value without wrapping about -pi and pi
    phase_unwrapped = np.unwrap(np.angle(freq_shifted))
    fig_phase_unwrapped = go.Figure()
    fig_phase_unwrapped.add_trace(go.Scatter(
        x=freqs_shifted,
        y=phase_unwrapped,
        mode='lines',
        name='Unwrapped Phase',
        line=dict(color='seagreen')
    ))
    fig_phase_unwrapped.update_layout(
        title="Frequency Domain: Unwrapped Phase",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Phase (radians)",
        template="plotly_white",
        hovermode="x unified"
    )

    # Show all
    fig_mag.show()
    fig_phase.show()
    fig_phase_unwrapped.show()
    fig_time.show()

In [22]:
# Interactive widgets
ui = interactive(
    plot_shifted_sinc,
    f_s=IntSlider(min=99, max=5000, step=100, value=1000, description="Sampling Freq (Hz)"),
    duration=FloatSlider(min=0, max=4.0, step=0.1, value=2.0, description="Duration (s)"),
    bandwidth=IntSlider(min=0, max=500, step=1, value=100, description="Bandwidth (Hz)"),
    time_shift=FloatSlider(min=-1.5, max=0.5, step=0.01, value=0.0, description="Shift t_0")
)
display(ui)

interactive(children=(IntSlider(value=1000, description='Sampling Freq (Hz)', max=5000, min=99, step=100), Flo…

Above, we applied a linear phase term, complex exponential $e^{j2\pi f t_0}$ to our frequency-domain signal, it results in a time-domain shift by $t_0$:
$$
X(f) \rightarrow X(f) \cdot e^{j2\pi f t_0}
$$

### Modulation Property of the Fourier Transform

The **modulation property** describes how multiplying a signal in one domain (time or frequency) by a complex exponential affects the other domain.

---

### Time-Domain Modulation (Shifting in Frequency)
If you **multiply** a time-domain signal by a complex exponential \( $e^{j 2\pi f_0 t}$ \):
$$
x(t) \cdot e^{j 2\pi f_0 t}
$$

Then its Fourier Transform is **shifted** in frequency:
$$
\mathcal{F} \left\{ x(t) \cdot e^{j 2\pi f_0 t} \right\} = X(f - f_0)
$$

Multiplying by \( $e^{j 2\pi f_0 t}$ \) **modulates** the signal, moving it up to frequency \( $f_0$ \)

---

### Frequency-Domain Modulation (Shifting in Time)

If you **multiply** a frequency-domain function by a complex exponential \( $e^{j 2\pi f t_0}$ \):
$$
X(f) \cdot e^{j 2\pi f t_0}
$$

Then its **inverse Fourier Transform** is:
$$
\mathcal{F}^{-1} \left\{ X(f) \cdot e^{j 2\pi f t_0} \right\} = x(t + t_0)
$$

Multiplying the spectrum by \( $e^{j 2\pi f t_0}$ \) shifts the time-domain signal by \( $t_0$ \)

---

### Summary Table

| Operation | Effect |
|----------|--------|
| Multiply $x(t)$ by $e^{j 2\pi f_0 t}$  | Shifts spectrum by $f_0$ : $X(f - f_0)$  |
| Multiply $X(f)$ by $e^{j 2\pi f t_0}$  | Shifts signal by $t_0$ : $x(t + t_0)$  |

---

### Example

Let:

$$
X(f) = \text{rect}\left( \frac{f}{B} \right)
$$

Then:

$$
x(t) = B \cdot \text{sinc}(Bt)
$$

Now multiply $X(f)$ by $e^{j 2\pi f t_0}$:

$$
X(f) \cdot e^{j 2\pi f t_0}
$$

Then the inverse Fourier transform becomes:

$$
x(t - t_0) = B \cdot \text{sinc}(B(t - t_0))
$$

---


In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import IntSlider, FloatSlider, interactive
from IPython.display import display

def plot_modulated_sinc(f_s, duration, bandwidth, freq_shift):
    N = int(f_s * duration)
    N = N + 1 if N % 2 == 0 else N

    # Time axis
    t = np.linspace(-duration / 2, duration / 2, N, endpoint=False)

    # Original time-domain sinc
    x_t = bandwidth * np.sinc(bandwidth * t)

    # Apply modulation (multiplying by e^{j 2π f₀ t})
    x_modulated = x_t * np.exp(1j * 2 * np.pi * freq_shift * t)

    # FFT of the modulated signal
    X_modulated = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(x_modulated)))
    freqs = np.fft.fftshift(np.fft.fftfreq(N, d=1/f_s))

    # --- Frequency Magnitude Plot ---
    fig_mag = go.Figure()
    fig_mag.add_trace(go.Scatter(x=freqs,
                                 y=np.abs(X_modulated),
                                 mode='lines',
                                 name='|X(f)|',
                                 line=dict(color='royalblue')))
    fig_mag.update_layout(
        title="Frequency Domain: Magnitude (after time modulation)",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Frequency Phase Plot (Unwrapped) ---
    fig_phase_unwrapped = go.Figure()
    fig_phase_unwrapped.add_trace(go.Scatter(
        x=freqs,
        y=np.unwrap(np.angle(X_modulated)),
        mode='lines',
        name='Unwrapped Phase',
        line=dict(color='seagreen')
    ))
    fig_phase_unwrapped.update_layout(
        title="Frequency Domain: Unwrapped Phase (after time modulation)",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Phase (radians)",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Time-Domain Plot (Real and Imag) ---
    fig_time = go.Figure()
    fig_time.add_trace(go.Scatter(x=t, y=np.real(x_modulated),
                                  mode='lines', name='Re{x(t) * e^{j2pif₀t}}',
                                  line=dict(color='darkorange')))
    fig_time.add_trace(go.Scatter(x=t, y=np.imag(x_modulated),
                                  mode='lines', name='Im{x(t) * e^{j2pif₀t}}',
                                  line=dict(color='mediumpurple')))
    fig_time.update_layout(
        title=f"Time Domain: Sinc(t) × e^{{j2pi f_0 t}}, f_0={freq_shift} Hz",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # Show all plots
    fig_mag.show()
    fig_phase_unwrapped.show()
    fig_time.show()

# Interactive controls
ui = interactive(
    plot_modulated_sinc,
    f_s=IntSlider(min=100, max=5000, step=100, value=1000, description="Sampling Freq (Hz)"),
    duration=FloatSlider(min=0.5, max=3.0, step=0.1, value=2.0, description="Duration (s)"),
    bandwidth=IntSlider(min=1, max=500, step=1, value=100, description="Bandwidth (Hz)"),
    freq_shift=FloatSlider(min=-300, max=300, step=5, value=0, description="Freq Shift $f_0$")
)
display(ui)


interactive(children=(IntSlider(value=1000, description='Sampling Freq (Hz)', max=5000, min=100, step=100), Fl…

## Cosine Modulation
We have seen the modulation of a signal by a complex exponential modulation. We could also perform modulation using a cosine modulation, $cos(2\pi f_0 t)$. 

The modulated signal can be written as such:
$$
x(t) = B \cdot sinc(Bt) \cdot cos(2\pi f_0 t)
$$

By Euler's formula, we can decompose $cos(2\pi f_0 t)$ into its Real and Complex components:
$$
cos(2\pi f_0 t) = \frac{1}{2}(e^{j2\pi f_0 t} + e^{-j2\pi f_0 t})
$$

Recall, that a complex exponential phase shift of $f_0$ applied to a signal in time domain would result in a frequency shift of $f_0$, that is $X(f-f_0)$. Hence,
$$
X(f) = \frac{1}{2} rect(\frac{f-f_0}{B}) + \frac{1}{2}rect(\frac{f+f_0}{B})
$$

As can be seen, we should be expecting two boxcars, one centred at $+f_0$ and another at $-f_0$

In [24]:
def compare_modulations(f_s=1000, duration=2.0, bandwidth=100, freq_shift=100):
    N = int(f_s * duration) 
    N = N + 1 if N % 2 == 0 else N

    # Time and frequency axes
    t = np.linspace(-duration / 2, duration / 2, N, endpoint=False)
    freqs = np.fft.fftshift(np.fft.fftfreq(N, d=1/f_s))

    # Base sinc signal
    sinc = bandwidth * np.sinc(bandwidth * t)

    # --- Complex modulation ---
    x_complex = sinc * np.exp(1j * 2 * np.pi * freq_shift * t)
    X_complex = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(x_complex)))

    # --- Real (cosine) modulation ---
    x_cosine = sinc * np.cos(2 * np.pi * freq_shift * t)
    X_cosine = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(x_cosine)))

    # --- Plot frequency magnitude comparison ---
    fig_freq = go.Figure()

    fig_freq.add_trace(go.Scatter(
        x=freqs,
        y=np.abs(X_complex),
        mode='lines',
        name='|X_complex(f)|',
        line=dict(color='orangered')
    ))

    fig_freq.add_trace(go.Scatter(
        x=freqs,
        y=np.abs(X_cosine),
        mode='lines',
        name='|X_cosine(f)|',
        line=dict(color='royalblue')
    ))

    fig_freq.update_layout(
        title=f"Frequency Domain: Complex vs Real Modulation (f₀ = {freq_shift} Hz)",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    # --- Plot time-domain comparison (real parts only) ---
    fig_time = go.Figure()

    fig_time.add_trace(go.Scatter(
        x=t,
        y=np.real(x_complex),
        mode='lines',
        name='Re{x_complex(t)}',
        line=dict(color='darkorange')
    ))

    fig_time.add_trace(go.Scatter(
        x=t,
        y=x_cosine,
        mode='lines',
        name='x_cosine(t)',
        line=dict(color='mediumblue')
    ))

    fig_time.update_layout(
        title="Time Domain: Real Part of Complex vs Cosine Modulated Sinc",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x unified"
    )

    fig_time.show()
    fig_freq.show()

In [25]:
ui = interactive(
    compare_modulations,
    f_s=IntSlider(min=500, max=5000, step=100, value=1000, description="Sampling Freq (Hz)"),
    duration=FloatSlider(min=0.5, max=3.0, step=0.1, value=2.0, description="Duration (s)"),
    bandwidth=IntSlider(min=10, max=300, step=10, value=100, description="Bandwidth (Hz)"),
    freq_shift=IntSlider(min=0, max=300, step=10, value=100, description="Carrier f₀ (Hz)")
)
display(ui)


interactive(children=(IntSlider(value=1000, description='Sampling Freq (Hz)', max=5000, min=500, step=100), Fl…